In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

In [2]:
def squared_euclidean_distance(x_1,x_2):
    sqd = np.sqrt(np.reshape(np.sum(x_1**2,1),newshape =(x_1.shape[0],1))+\
    np.reshape(np.sum(x_2**2,1),newshape =(1,x_2.shape[0]))-\
    2*(x_1@x_2.T))
    return sqd

def cartesian_dist(x_1,x_2):
    return np.concatenate([
        np.tile(x_1[:,0] - np.reshape(x_2[:,0],[x_2[:,0].shape[0],1]),[1,3]),
        np.tile(x_1[:,1] - np.reshape(x_2[:,1],[x_2[:,1].shape[0],1]),[1,3]),
        np.tile(x_1[:,2] - np.reshape(x_2[:,2],[x_2[:,2].shape[0],1]),[1,3])],axis = 0) 

def cov_gradients(dist_tiled):
    
    condition1 = 0
    a = (h_u*h_v)
    b = dist_tiled**2

    t1 =  np.divide(a, b, out=np.zeros_like(a),casting='unsafe', where=b!=0)
    t2 = np.where(dist_tiled < a_T,(-c_o_T*((-14/a_T**2)+
                 105/4*dist_tiled/a_T**3 -
                 35/2 * dist_tiled**3 / a_T **5 +
                 21 /4 * dist_tiled**5/a_T**7)),0)+\
          np.where(dist_tiled < a_T,(c_o_T * 7 * (9 * dist_tiled ** 5 -
                      20 * a_T ** 2 * dist_tiled ** 3 +
                      15 * a_T ** 4 * dist_tiled -
                      4 * a_T ** 5) / (2 * a_T ** 7)),0)

    # when we do the covariance of Gx with Gx, Gy with Gy and so on, quation B9 in Gempy paper
    t3 = perpendicularity_matrix * \
          np.where(dist_tiled < a_T,(c_o_T * ((-14 / a_T ** 2) + 105 / 4 * dist_tiled / a_T ** 3 -
                   35 / 2 * dist_tiled ** 3 / a_T ** 5 +
                   21 / 4 * dist_tiled ** 5 / a_T ** 7)),0)
    t4 = 1/3*np.eye(dist_tiled.shape[0])

    condition2 = t1 * t2 - t3 + t4

    C_G = np.where(dist_tiled==0, condition1, condition2) ## adding nugget effect
    return C_G

def set_rest_ref_matrix(number_of_points_per_surface):
    ref_layer_points = np.repeat(np.stack([layer1[-1]],axis = 0),repeats=number_of_points_per_surface-1,axis = 0)
    rest_layer_points = np.concatenate([layer1[0:-1]],axis = 0)
    return ref_layer_points,rest_layer_points

def cov_interface(ref_layer_points,rest_layer_points):
    sed_rest_rest = squared_euclidean_distance(rest_layer_points,rest_layer_points)
    sed_ref_rest = squared_euclidean_distance(ref_layer_points,rest_layer_points)
    sed_rest_ref = squared_euclidean_distance(rest_layer_points,ref_layer_points)
    sed_ref_ref = squared_euclidean_distance(ref_layer_points,ref_layer_points)
    
    C_I = c_o_T*(\
                (np.where(sed_rest_rest < a_T, (1 - 7 * (sed_rest_rest / a_T) ** 2 +\
                 35 / 4 * (sed_rest_rest / a_T) ** 3 -\
                 7 / 2 * (sed_rest_rest / a_T) ** 5 +\
                 3 / 4 * (sed_rest_rest / a_T) ** 7),0)) -\
                (np.where(sed_ref_rest < a_T, (1 - 7 * (sed_ref_rest / a_T) ** 2 +\
                  35 / 4 * (sed_ref_rest / a_T) ** 3 -\
                  7 / 2 * (sed_ref_rest / a_T) ** 5 +\
                  3 / 4 * (sed_ref_rest / a_T) ** 7),0)) -\
                (np.where(sed_rest_ref < a_T, (1 - 7 * (sed_rest_ref / a_T) ** 2 +\
                  35 / 4 * (sed_rest_ref / a_T) ** 3 -\
                  7 / 2 * (sed_rest_ref / a_T) ** 5 +\
                  3 / 4 * (sed_rest_ref / a_T) ** 7),0)) +\
                 (np.where(sed_ref_ref < a_T, (1 - 7 * (sed_ref_ref / a_T) ** 2 +\
                  35 / 4 * (sed_ref_ref / a_T) ** 3 -\
                  7 / 2 * (sed_ref_ref / a_T) ** 5 +\
                  3 / 4 * (sed_ref_ref / a_T) ** 7),0)))
    
    return C_I

def cartesian_dist_no_tile(x_1,x_2):
    return np.concatenate([
        np.transpose((x_1[:,0] - np.reshape(x_2[:,0],[x_2.shape[0],1]))),
        np.transpose((x_1[:,1] - np.reshape(x_2[:,1],[x_2.shape[0],1]))),
        np.transpose((x_1[:,2] - np.reshape(x_2[:,2],[x_2.shape[0],1])))],axis = 0) 

def cov_interface_gradients(hu_rest,hu_ref):
    C_GI = hu_rest*np.where(sed_dips_rest < a_T,(- c_o_T * ((-14 / a_T ** 2) + 105 / 4 * sed_dips_rest / a_T ** 3 -
                                  35 / 2 * sed_dips_rest ** 3 / a_T ** 5 +
                                  21 / 4 * sed_dips_rest ** 5 / a_T ** 7)),0)-\
    hu_ref*np.where(sed_dips_ref < a_T,(-c_o_T * ((-14 / a_T ** 2) + 105 / 4 * sed_dips_ref / a_T ** 3 -
                                  35 / 2 * sed_dips_ref ** 3 / a_T ** 5 +
                                  21 / 4 * sed_dips_ref ** 5 / a_T ** 7)),0)
    return C_GI

def safe_arange(start, stop, step):
    return step * np.arange(start / step, stop / step)

def perpendicularity(G_1):
    a = np.concatenate([np.ones([G_1.shape[0],G_1.shape[0]]),np.zeros([G_1.shape[0],G_1.shape[0]]),np.zeros([G_1.shape[0],G_1.shape[0]])],axis = 1)
    b = np.concatenate([np.zeros([G_1.shape[0],G_1.shape[0]]),np.ones([G_1.shape[0],G_1.shape[0]]),np.zeros([G_1.shape[0],G_1.shape[0]])],axis = 1)
    c = np.concatenate([np.zeros([G_1.shape[0],G_1.shape[0]]),np.zeros([G_1.shape[0],G_1.shape[0]]),np.ones([G_1.shape[0],G_1.shape[0]])],axis = 1)
    return np.concatenate([a,b,c],axis = 0)    



In [3]:

G_1 = np.array([[5,6,0],[5,4,0]])

G_1_o = np.array([[0,1,0],[0,1,0]])

a_T = 5

layer1 = np.array([[8.5,2,0],[7,4,0],[5.5,5.4,0],[4,5.4,0],[2.5,4,0],[1,2,0]])

c_o_T = a_T**2/14/3

G_1_tiled = np.tile(G_1,[3,1])

h_u = cartesian_dist(G_1,G_1)
h_v = h_u.T

perpendicularity_matrix = perpendicularity(G_1)

dist_tiled = squared_euclidean_distance(G_1_tiled,G_1_tiled)

dist_tiled = dist_tiled + np.eye(dist_tiled.shape[0])

C_G = cov_gradients(dist_tiled)

number_of_layer = 1

number_of_points_per_surface = np.array([layer1.shape[0]])

ref_layer_points,rest_layer_points = set_rest_ref_matrix(number_of_points_per_surface)

sed_rest_rest = squared_euclidean_distance(rest_layer_points,rest_layer_points)
sed_ref_rest = squared_euclidean_distance(ref_layer_points,rest_layer_points)
sed_rest_ref = squared_euclidean_distance(rest_layer_points,ref_layer_points)
sed_ref_ref = squared_euclidean_distance(ref_layer_points,ref_layer_points)

C_I = cov_interface(ref_layer_points,rest_layer_points)

sed_dips_rest = squared_euclidean_distance(G_1_tiled,rest_layer_points)
sed_dips_ref = squared_euclidean_distance(G_1_tiled,ref_layer_points)

hu_rest = cartesian_dist_no_tile(G_1,rest_layer_points)
hu_ref = cartesian_dist_no_tile(G_1,ref_layer_points)


C_GI = cov_interface_gradients(hu_rest,hu_ref)
C_IG = C_GI.T

xx = safe_arange(0,10,0.5)
yy = safe_arange(0,10,0.5)
zz = safe_arange(0,10,0.5)
XX,YY,ZZ = np.meshgrid(xx,yy,zz)
X = (np.reshape(XX,[-1])).T
Y = (np.reshape(YY,[-1])).T
Z = (np.reshape(ZZ,[-1])).T
grid = np.stack([X,Y,Z],axis = 1)


hu_Simpoints = cartesian_dist_no_tile(G_1,grid)
sed_dips_SimPoint = squared_euclidean_distance(G_1_tiled,grid)
sed_rest_SimPoint = squared_euclidean_distance(rest_layer_points,grid)
sed_ref_SimPoint = squared_euclidean_distance(ref_layer_points,grid)


K = np.concatenate([np.concatenate([C_G,C_GI],axis = 1),np.concatenate([C_IG,C_I],axis = 1)],axis = 0)

n = G_1.shape[0]
sub_x = np.tile(np.array([[0,0,0]]),[n,1])
sub_y = np.tile(np.array([[0,1,0]]),[n,1])
sub_z = np.tile(np.array([[0,0,1]]),[n,1])
sub_block1 = np.concatenate([sub_x,sub_y,sub_z],0)

sub_x_2 = 0*np.reshape(1*G_1[:,0],[n,1])
sub_y_2 = np.reshape(1*G_1[:,1],[n,1])
sub_z_2 = np.reshape(1*G_1[:,2],[n,1])
sub_block2 = np.concatenate([sub_x_2, sub_y_2, sub_z_2], 0)

sub_xy = np.reshape(np.concatenate([G_1[:, 1],0*G_1[:, 0]], 0), [2 * n, 1])
sub_xy = np.pad(sub_xy, [[0, n], [0, 0]])

sub_xz = np.concatenate([np.pad(np.reshape(0*G_1[:, 2], [n, 1]), [[0, n], [0, 0]]), np.reshape(0*G_1[:, 0], [n, 1])], 0)

sub_yz = np.reshape(np.concatenate([0*G_1[:, 2], G_1[:, 1]], 0), [2 * n, 1])
sub_yz = np.pad(sub_yz, [[n, 0], [0, 0]])

sub_block3 = np.concatenate([sub_xy, sub_xz, sub_yz], 1)

U_G = np.concatenate([sub_block1, sub_block2, sub_block3], 1)

U_I = -np.stack([0*(rest_layer_points[:, 0] - ref_layer_points[:, 0]), 
                 (rest_layer_points[:, 1] - ref_layer_points[:, 1]),
                 (rest_layer_points[:, 2] - ref_layer_points[:, 2]),
                 0*(rest_layer_points[:, 0] ** 2 - ref_layer_points[:, 0] ** 2),
                 (rest_layer_points[:, 1] ** 2 - ref_layer_points[:, 1] ** 2),
                 (rest_layer_points[:, 2] ** 2 - ref_layer_points[:, 2] ** 2),
                 0*(rest_layer_points[:, 0] * rest_layer_points[:, 1] - ref_layer_points[:, 0] * ref_layer_points[:, 1]),
                 0*(rest_layer_points[:, 0] * rest_layer_points[:, 2] - ref_layer_points[:, 0] * ref_layer_points[:, 2]),
                 (rest_layer_points[:, 1] * rest_layer_points[:, 2] - ref_layer_points[:, 1] * ref_layer_points[:, 2])], 1)

length_of_CG = C_G.shape[1]
length_of_CGI = C_GI.shape[1]

U_G = U_G[:length_of_CG, :3]
U_I = U_I[:length_of_CGI, :3]
D_U = np.concatenate([U_G,U_I],0)

D_X = np.zeros([G_1.shape[0],1])
D_Y = np.ones([G_1.shape[0],1])
D_Z = np.cos(G_1[:,2]).reshape(G_1.shape[0],1)
D_I = ((np.sin(ref_layer_points[:,2])+ref_layer_points[:,1]+0*ref_layer_points[:,0]) - (np.sin(rest_layer_points[:,2])+rest_layer_points[:,1]+0*rest_layer_points[:,0])).reshape(5,1)


D_E = np.concatenate([D_X , D_Y , D_Z , D_I],axis=0)
D_E_T = D_E.T


D = np.concatenate([D_U,D_E],axis=1)
D = D_E
D_T = D.T 

# build zero matrix
zero_matrix = np.zeros([D.shape[1],D.shape[1]])


# concatenate drift matrix and kriging matrix
K_D = np.concatenate([np.concatenate([K,D],axis = 1),np.concatenate([D_T,zero_matrix],axis = 1)],axis = 0)

# build right side matrix of cokriging system
bk = np.concatenate([G_1_o[:,0],G_1_o[:,1],G_1_o[:,2],np.zeros(K_D.shape[0]-G_1.shape[0]*3)],axis = 0)
bk = np.reshape(bk,newshape = [bk.shape[0],1])

# solve kriging weight
w = np.linalg.lstsq(K_D,bk)[0]
# gradient contribution
sigma_0_grad = w[:G_1.shape[0]*3] * (-hu_Simpoints*(sed_dips_SimPoint < a_T)*(- c_o_T * ((-14 / a_T ** 2) + 105 / 4 * sed_dips_SimPoint / a_T ** 3 - 35 / 2 * sed_dips_SimPoint ** 3 / a_T ** 5 + 21 / 4 * sed_dips_SimPoint ** 5 / a_T ** 7)))

sigma_0_grad = np.sum(sigma_0_grad,axis=0)
# surface point contribution
sigma_0_interf = -w[G_1.shape[0]*3:-1]*(c_o_T  * ((sed_rest_SimPoint < a_T)*(1 - 7 * (sed_rest_SimPoint / a_T) ** 2 +
            35 / 4 * (sed_rest_SimPoint / a_T) ** 3 -
            7 / 2 * (sed_rest_SimPoint / a_T) ** 5 +
            3 / 4 * (sed_rest_SimPoint / a_T) ** 7) -
            (sed_ref_SimPoint < a_T)*(1 - 7 * (sed_ref_SimPoint / a_T) ** 2 +
            35 / 4 * (sed_ref_SimPoint / a_T) ** 3 -
            7 / 2 * (sed_ref_SimPoint / a_T) ** 5 +
            3 / 4 * (sed_ref_SimPoint / a_T) ** 7)))
sigma_0_interf = np.sum(sigma_0_interf,axis = 0)

external_drift = (np.sin(grid[:,2]) + grid[:,1]+ 0*grid[:,0]) * (w[-1]).T

interpolate_result = 1*sigma_0_grad+1*sigma_0_interf + 1* external_drift

intp = np.reshape(interpolate_result,[20,20,20]) # reshape the result to matrix shape
intp_c = intp





C:\Users\yangj\AppData\Local\Temp\ipykernel_7744\348692551.py:178: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  w = np.linalg.lstsq(K_D,bk)[0]


In [4]:
import pyvista as pv
import pyvistaqt as pvqt
def plot_3D(grid,value,mode='block',surfaces_nr=10):
    x = grid[:,0]
    y = grid[:,1]
    z = grid[:,2]

    grid = pv.UniformGrid(dims=(20,20,20), spacing=(0.5,0.5,0.5),origin=(0,0,0))
    p = pvqt.BackgroundPlotter()
    layer1_ = layer1
    if mode == 'block':
        p.add_mesh(grid,scalars=value.flatten(),show_edges=False,style='surface')
        p.add_axes()
        p.show()

    elif mode == 'surfaces':
        grid['vol'] = value.ravel()
        lvl_t=value[np.where((XX==layer1[0,0])&(YY==layer1[0,1])&(ZZ==layer1[0,2]))]
        contours_1 = grid.contour(np.linspace(value.min(),value.max(),surfaces_nr))
        contours_2 = grid.contour(lvl_t)
        #p.add_mesh(contours_1, show_scalar_bar=True, label='scalar_field',style='surface')
        p.add_mesh(contours_2, show_scalar_bar=True, label='scalar_field',style='surface',color='red')

    p.add_points(layer1[:,[2,0,1]], render_points_as_spheres=True,
                      point_size=20.0)
    orientation1 = pv.Arrow(start=(G_1[:,[2,0,1]][0]),direction=(G_1_o[:,[2,0,1]][0]))
    orientation2 = pv.Arrow(start=(G_1[:,[2,0,1]][1]),direction=(G_1_o[:,[2,0,1]][1]))
    p.add_mesh(orientation1)
    p.add_mesh(orientation2)
    p.add_axes()
    p.show()

In [5]:
plot_3D(grid,intp,mode='surfaces',surfaces_nr=10)